# Testar SMHI:s API

In [31]:

# !pip install smhi-open-data
from smhi_open_data import SMHIOpenDataClient, Parameter


import requests
from datetime import datetime
import pandas as pd
import json

# Get the data from the API
url = 'https://opendata-download-metobs.smhi.se/api'
endpoint = '/version/1.0/parameter/1/station/159880/period/latest-months.json'
r = requests.get(url + endpoint)

# url = 'https://opendata-download-metobs.smhi.se/api/version/latest/parameter/4.json'
# r = requests.get(url)

print(r.json())

# r = requests.get(url + endpoint)
data = r.json()

# data = json.dumps(data, indent=4, sort_keys=True)

data = pd.json_normalize(data)

# Convert the data to a pandas DataFrame
# data = pd.read_json(data)

print(data.head())

data.columns

data.loc[0,'link']

{'key': 'latest-months', 'updated': 1670284800000, 'title': 'Lufttemperatur - Arvidsjaur A - Data från senaste fyra månaderna: Ladda ner data', 'summary': 'Här laddar du ner din datamängd i CSV-format. Inställningar i din Webb-läsare styr hur data presenteras. Om du väljer att arbeta i Excel kan dessa filer vara för stora och kortas av.', 'from': 1659052801000, 'to': 1670360400000, 'link': [{'rel': 'period', 'type': 'application/json', 'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/1/station/159880/period/latest-months.json'}, {'rel': 'period', 'type': 'application/xml', 'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/1/station/159880/period/latest-months.xml'}, {'rel': 'period', 'type': 'application/atom+xml', 'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/1/station/159880/period/latest-months.atom'}, {'rel': 'station', 'type': 'application/atom+xml', 'href': 'https://opendata-download-metobs.smhi.se

[{'rel': 'period',
  'type': 'application/json',
  'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/1/station/159880/period/latest-months.json'},
 {'rel': 'period',
  'type': 'application/xml',
  'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/1/station/159880/period/latest-months.xml'},
 {'rel': 'period',
  'type': 'application/atom+xml',
  'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/1/station/159880/period/latest-months.atom'},
 {'rel': 'station',
  'type': 'application/atom+xml',
  'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/1/station/159880.atom'}]

## Använder färdigt bibliotek för smhi-api
`pip install smhi-open-data`

In [32]:
from smhi_open_data import SMHIOpenDataClient, Parameter

# Get 10 stations
client = SMHIOpenDataClient()

# Get all stations
stations = client.get_stations()

# Get closest station
closest_station = client.get_closest_station(
    latitude=55.707722,
    longitude=12.562119)

# Get available parameters
parameters = client.list_parameters()

# Get available parameters at station
parameters_station = client.get_station_parameters(station_id=173010)

# Get temperature observations from available stations from past hour
observations = client.get_latest_observations(
    parameter=Parameter.TemperaturePast1h)

In [33]:
# print(observations)
print(observations[0])
for key in closest_station.keys():
    print(key)

print(closest_station['id'])
print(closest_station['name'])

# Get temperature observations from closest station from past hour
observations = client.get_latest_observations(
    parameter=Parameter.TemperaturePast1h)

print(f"Temperature at {closest_station['name']} is {observations[0]['value']}.")

{'parameter_id': 1, 'timestamp': 1670360400000, 'value': -7.0, 'station': '188790'}
name
owner
ownerCategory
measuringStations
id
height
latitude
longitude
active
from
to
key
updated
title
summary
link
52550
Ven
Temperature at Ven is -7.0.


## Använder länk från `link`-fältet i API-dokumentationen	

Vi kan då få ut data från angiven station i json, xml eller csv-format.

In [34]:
from datetime import datetime

html_smhi = 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/1/station/159880/period/latest-months/data.json'
r = requests.get(html_smhi)
data = r.json()

# read data into a DataFrame from json response
df = pd.DataFrame(data['value'])

def convert_to_datetime(x):
    return datetime.fromtimestamp(x/1000)

timstamp = df['date'][1]

time = convert_to_datetime(timstamp)

print(time)

df['date'] = df['date'].apply(convert_to_datetime)
df


2022-07-29 04:00:00


,date,value,quality
0,2022-07-29 03:00:00,7.6,G
1,2022-07-29 04:00:00,7.1,G
2,2022-07-29 05:00:00,8.1,G
3,2022-07-29 06:00:00,8.3,G
4,2022-07-29 07:00:00,9.2,G
...,...,...,...
3136,2022-12-06 19:00:00,-6.7,G
3137,2022-12-06 20:00:00,-7.0,G
3138,2022-12-06 21:00:00,-7.1,G
3139,2022-12-06 22:00:00,-7.1,G
